[View in Colaboratory](https://colab.research.google.com/github/JozeeLin/google-tensorflow-exercise/blob/master/TF_Learn_AND_TF_contrib.ipynb)

## 分布式Estimator

本节介绍Estimator的分布式特性、自定义模型的用法、Estimator的架构，并介绍怎样建立自己的分布式学习Estimator

In [0]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import learn

#自定义模型，参数feature标识数据的特征，target表示数据特征每一行的目标或分类的标识
def my_model(features,target):
  target = tf.one_hot(target, 3,1,0) #利用tf.one_hot对targets进行独热编码
  #用layers.stack叠加多层layers.fully_connected完全连接的深度神经网络。(进行特征工程)
  features = layers.stack(features, layers.fully_connected, [10,20,10]) 
  prediction,loss = tf.contrib.learn.models.logistic_regression_zero_init(features,target)#定义初始参数为0的逻辑斯蒂回归模型，从而获取预测值和损失值
  
  #使用contrib.layers.optimize_loss函数对损失值进行优化，根据不同需要选择不同的优化函数和学习率。
  #train_op为训练算子，在每次训练迭代时会被用来优化模型的参数和决定模型发展的方向
  train_op = tf.contrib.layers.optimize_loss(
      loss,tf.contrib.framework.get_global_step(),optimizer='Adagrad',
      learning_rate=0.1
  )
  
  return {'class':tf.argmax(prediction,1),'prob':prediction},loss,train_op

In [0]:
from sklearn import datasets
from sklearn import model_selection

In [5]:
iris = datasets.load_iris()
x_train,x_test,y_train,y_test = model_selection.train_test_split(iris.data,
                                                                iris.target,
                                                                test_size=0.2,
                                                                random_state=35)
#把定义好的my_model直接放进learn.Estimator就可以使用Scikit-learn风格的fit和predict函数，也能够直接进行分布式模型训练
classifier = learn.Estimator(model_fn=my_model)
classifier.fit(x_train,y_train,steps=700)

predictions = classifier.predict(x_test)

Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff9bd9849b0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpj_5i1rig'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into

INFO:tensorflow:loss = 1.0986123085021973, step = 0
INFO:tensorflow:global_step/sec: 207.525
INFO:tensorflow:loss = 0.28252872824668884, step = 100 (0.487 sec)
INFO:tensorflow:global_step/sec: 218.021
INFO:tensorflow:loss = 0.0776234120130539, step = 200 (0.460 sec)
INFO:tensorflow:global_step/sec: 206.538
INFO:tensorflow:loss = 0.07313373684883118, step = 300 (0.481 sec)
INFO:tensorflow:global_step/sec: 209.805
INFO:tensorflow:loss = 0.06207158416509628, step = 400 (0.480 sec)
INFO:tensorflow:global_step/sec: 208.638
INFO:tensorflow:loss = 0.05924055352807045, step = 500 (0.479 sec)
INFO:tensorflow:global_step/sec: 209.573
INFO:tensorflow:loss = 0.057882893830537796, step = 600 (0.476 sec)
INFO:tensorflow:Saving checkpoints for 700 into /tmp/tmpj_5i1rig/model.ckpt.
INFO:tensorflow:Loss for final step: 0.05631427839398384.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
availabl

## TF.Contrib

TF.Contrib是TensorFlow里很重要的一个部分，很大一部分开源社区的贡献都被集中在这里，特别是一些新功能，由于都是一些刚贡献的功能，谷歌会将这些代码暂时放在这里，由谷歌内部及外部的用户一次测试，等它们的API都比较稳定的时候，会被移到TensorFlow的核心模块。

这个模块里提供了机器学习需要的大部分功能，包括统计分布、机器学习层、优化函数、指标，等等。

### 统计分布
TF.contrib.distributions模块里有许多的统计分布。这些统计分布大多数都是统计研究和应用中经常用到的，也是各种统计及机器学习模型的即使，许多的概率模型和图形模型都非常依赖这些统计分布。

### layer模块
contrib.layer包含了机器学习算法所需的各种各样的成分和部件，例如卷积层、批标准化层、机器学习指标、优化函数、初始器、特征列、等。有了这些基础的建设部件，我们可以高效的建立复杂的机器学习及机器学习系统。本章将介绍这个模块里一些主要的部件，来帮助理解TensorFlow的各种可能性及灵活性。